# Soluciones al examen de OPI 

In [86]:
# paquetes necesarios para este examen
import pandas as pd
from datetime import datetime # para que Python reconozca las fechas
import numpy as np
import math
import matplotlib.pyplot as plt

1. Los datos son de calidad cuando sirven para una finalidad prevista, basados en las siguientes características:
* No debe haber huecos, ni variables repetidas.
* La información debe ser relevante para cumplir los objetivos, por ejemplo, si estoy haciendo un análisis sobre los delitos en la ciudad de México, no me sirve tener información sobre el peso de los habitantes de la ciudad.
* Confiabilidad: hay que asegurarse que los datos no sean ambiguos, vagos o que contengan información contradictoria, por ejemplo, si tengo información sobre los delitos de una alcaldía en dos columnas y cada una con información diferente.
* Los datos tienen que ser colectados en el tiempo correcto, hay que asegurarse que no sean obsoletos, tomando de nuevo el análisis de los delitos de la CDMX, no tendría sentido que sólo tenga información de hace 30 años.

2. Datos a la alza y a la baja.

In [25]:
# importamos la base de datos
# encoding='utf-8' es para que al importarse la base de datos, reconozca caracteres 
# como la ñ.
# dtype='unicode' es para evitar un warning por low_memory ya que python estaba 
# adivinando el tipo de objeto (dtype) de cada columna

datos = pd.read_csv('/home/lorena/Escritorio/OPI/carpetas_completa_febrero_2022.csv', encoding='utf-8', dtype='unicode')

Para que las columnas fecha_hechos y fecha_inicia tal cual sean reconocidas como fechas, se utiliza la función de pandas **to_datetime**.

In [30]:
datos['fecha_hechos'] = pd.to_datetime(datos['fecha_hechos'], format = "%Y %m %d %H:%M:%S")
datos['fecha_inicio'] = pd.to_datetime(datos['fecha_inicio'], format = "%Y %m %d %H:%M:%S")

In [28]:
datos.columns

Index(['ao_hechos', 'mes_hechos', 'fecha_hechos', 'ao_inicio', 'mes_inicio',
       'fecha_inicio', 'delito', 'fiscalia', 'agencia', 'unidad_investigacion',
       'categoria_delito', 'calle_hechos', 'calle_hechos2', 'colonia_hechos',
       'alcaldia_hechos', 'competencia', 'longitud', 'latitud', 'tempo'],
      dtype='object')

A continuación, descarto a aquellas filas en las que el año de los hechos sea igual a **nan**:

In [108]:
datos = datos[np.logical_not(pd.isnull(datos['ao_hechos']))]

In [112]:
# arreglo con los nombres de los delitos que aparecen en la base de datos, sin repetir
delitos = datos['delito'].unique()

In [113]:
len(delitos)

347

In [110]:
# arreglo con los años de los hechos que aparecen en la base de datos, sin repetir
año_hechos = datos['ao_hechos'].unique()

In [111]:
año_hechos

array(['2016', '2015', '2012', '2014', '2011', '2008', '2006', '2010',
       '2013', '2005', '2009', '1955', '2004', '2002', '1998', '1997',
       '1965', '2007', '2000', '1987', '1999', '1956', '2001', '1996',
       '1986', '1915', '1988', '2017', '2003', '1991', '1993', '1977',
       '1984', '1995', '1994', '1990', '1992', '1978', '1989', '1972',
       '2018', '2019', '1974', '1970', '1980', '1976', '1983', '1985',
       '1966', '1969', '1981', '1960', '1982', '1975', '1954', '1973',
       '1964', '1968', '1963', '1971', '1958', '1942', '1930', '1979',
       '1906', '1962', '1957', '2020', '1950', '1952', '2021', '1967',
       '1961', '1917', '2022'], dtype=object)

In [57]:
# arreglo con los años de inicio que aparecen en la base de datos, sin repetir
año_inicio = datos['ao_inicio'].unique()

In [58]:
año_inicio

array(['2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype=object)

In [59]:
datos['ao_inicio'] 

2016
2017
2018
2019
2020
2021
2022


In [60]:
# primer delito de la lista
delitos[0]

'DAÑO EN PROPIEDAD AJENA INTENCIONAL'

In [114]:
# selección de la base de datos del primer delito de la lista
primer_delito = datos[datos['delito'] == delitos[0]]

In [115]:
primer_delito.shape

(15005, 19)

In [118]:
# Agrupación de la sub base anterior de acuerdo al año de los hechos
# Después de la agrupación, se contó el número de repeticiones del delito por cada año
cuenta_primer_delito = primer_delito.groupby('ao_hechos').size()

In [119]:
# Se descartó el año 2022 para que no afecte en la medida del alza
cuenta_primer_delito[:-1]

ao_hechos
1972       1
1986       1
1989       1
1996       1
2001       2
2005       1
2006       5
2007       1
2008       3
2010       3
2011       3
2012       4
2013       9
2014      22
2015     122
2016    2147
2017    2916
2018    2821
2019    2401
2020    2037
2021    2201
dtype: int64

In [116]:
primer_delito.sort_values('ao_hechos')

,ao_hechos,mes_hechos,fecha_hechos,ao_inicio,mes_inicio,fecha_inicio,delito,fiscalia,agencia,unidad_investigacion,categoria_delito,calle_hechos,calle_hechos2,colonia_hechos,alcaldia_hechos,competencia,longitud,latitud,tempo
1303560,1972,Febrero,1972-02-27 20:26:00,2021,Septiembre,2021-09-25 21:06:29,DAÑO EN PROPIEDAD AJENA INTENCIONAL,AGENCIA DE DENUNCIA DIGITAL,CEN-1,Unidad de Investigación 1 S/D,DELITO DE BAJO IMPACTO,1 LEONARDO CHÁVEZ,COMBATE DE CELAYA,UNIDAD VICENTE GUERRERO,IZTAPALAPA,FUERO COMUN,-99.0568309004107,19.356887769104,NaN
1375229,1986,Noviembre,1986-11-01 12:00:00,2022,Enero,2022-01-18 10:52:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN CUAUH...,CUH-4,UI-1SD,DELITO DE BAJO IMPACTO,JOSE JOAQUIN HERRARA,MANUEL DOBLADO,CENTRO,CUAUHTEMOC,FUERO COMUN,-99.1249658201799,19.4387925430714,NaN
1372482,1989,Agosto,1989-08-02 13:59:00,2022,Enero,2022-01-12 21:38:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL,AGENCIA DE DENUNCIA DIGITAL,CEN-1,Unidad de Investigación 1 S/D,DELITO DE BAJO IMPACTO,COSTA,REMOLINO,AMPLIACIÓN ALPES,ALVARO OBREGON,FUERO COMUN,-99.2105220298719,19.3572110133824,NaN
1107739,1996,Noviembre,1996-11-12 01:54:00,2020,Noviembre,2020-11-12 04:32:18,DAÑO EN PROPIEDAD AJENA INTENCIONAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAP...,IZP-5,UI-3CD,DELITO DE BAJO IMPACTO,ERMITA IZTAPALAPA,NaN,SANTA MARTHA ACATITLA,IZTAPALAPA,FUERO COMUN,-99.0021821342655,19.3576916189887,NaN
345645,2001,Marzo,2001-03-16 22:03:00,2017,Marzo,2017-03-29 21:23:32,DAÑO EN PROPIEDAD AJENA INTENCIONAL,INVESTIGACIÓN EN IZTAPALAPA,IZP-9,UI-3SD,DELITO DE BAJO IMPACTO,AIDA,NaN,2A. AMPLIACIÓN SANTIAGO ACAHUALTEPEC,IZTAPALAPA,NaN,-99.014472697062,19.3484287397097,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379821,2022,Enero,2022-01-11 10:00:00,2022,Enero,2022-01-26 11:35:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN MILPA...,UAT-MIL-1,UI-1SD,DELITO DE BAJO IMPACTO,PRIMER CALLEJON INDEENDENCIA NUMERO 03,NaN,PANCHIMALCO (SAN PEDRO ATOCPAN),MILPA ALTA,FUERO COMUN,-99.0452243102281,19.2027794861998,NaN
1379736,2022,Enero,2022-01-26 07:30:00,2022,Enero,2022-01-26 08:39:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN CUAUH...,CUH-2,UI-3CD,DELITO DE BAJO IMPACTO,RIBERA DE SAN COSME,NaN,SANTA MARIA LA RIBERA,CUAUHTEMOC,FUERO COMUN,-99.1569104798388,19.4405834492559,NaN
1379095,2022,Enero,2022-01-23 17:00:00,2022,Enero,2022-01-25 09:50:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN CUAUH...,UAT-CUH-8,UI-1SD,DELITO DE BAJO IMPACTO,JOSE ANTONIO TORRES,NaN,ASTURIAS,CUAUHTEMOC,FUERO COMUN,-99.1341755196186,19.4036526130825,NaN
1380318,2022,Enero,2022-01-26 17:30:00,2022,Enero,2022-01-27 00:15:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN XOCHI...,UAT-XO-2,UI-1SD,DELITO DE BAJO IMPACTO,CERRADA VIOLETA,NaN,BARRIO 18,XOCHIMILCO,FUERO COMUN,-99.1121988099598,19.2813028171117,NaN
